In [2]:
import spacy
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, T5ForConditionalGeneration, AdamW, get_scheduler, DataCollatorWithPadding
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

d:\learnx.ai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
max_source_length = 512
max_target_length = 128
prefix_1 = "answer: "
prefix_2 = " context: "
prefix_3 = "question: "
checkpoint = "t5-small"

In [4]:

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess_function(examples):
  input_1 = [ prefix_1 + i[0] for i in examples["answers.text"] ]
  input_2 = [ prefix_2 + i for i in examples["context"] ]
  input = [input_1[i] + input_2[i] for i in range(len(examples["context"]))]
  inputs = [input[0] for i in range(len(examples["context"]))]
  model_inputs = tokenizer(inputs, padding="longest", max_length = max_source_length , truncation = True, return_tensors = "pt")
  labels = tokenizer([prefix_3 + i for i in examples["question"]], padding="longest", max_length = max_target_length ,truncation = True, return_tensors = "pt")
  labels["input_ids"][labels["input_ids"]==tokenizer.pad_token_id] = -100
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [5]:
squad = load_dataset("squad")
squad = squad.flatten()
squad["train"][0]["answers.text"]

['Saint Bernadette Soubirous']

In [6]:
squad_tokenized = squad.map(preprocess_function, batched=True)

In [7]:
squad_tokenized = squad_tokenized.remove_columns(["id","title","question","context","answers.text","answers.answer_start"])
squad_tokenized.set_format("torch")
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = checkpoint)

In [8]:
squad_tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10570
    })
})

In [9]:
train_dataloader = DataLoader(squad_tokenized["train"], batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(squad_tokenized["validation"], batch_size=8, collate_fn=data_collator)

In [10]:
for batch in train_dataloader:
    break
print({k:v.shape for k,v in batch.items()})


{'input_ids': torch.Size([8, 192]), 'attention_mask': torch.Size([8, 192]), 'labels': torch.Size([8, 41])}


d:\learnx.ai\.venv\lib\site-packages\transformers\data\data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [11]:
for batch in train_dataloader:
  break
print(tokenizer.decode(batch["input_ids"][0]))
fake_labels = np.where(batch["labels"][0]!=-100, batch["labels"][0], tokenizer.pad_token_id)
print(tokenizer.decode(fake_labels))

answer: Saint Bernadette Soubirous context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.</s>
question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [12]:
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(4.2687, grad_fn=<NllLossBackward0>) torch.Size([8, 41, 32128])


In [13]:
len(train_dataloader)

10950

In [14]:
optimizer = AdamW(model.parameters(), lr=1e-4)
loss = outputs.loss
loss.backward()
optimizer.step()
optimizer.zero_grad()

d:\learnx.ai\.venv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
num_epochs = 20
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

cuda


In [17]:
optimizer = AdamW(model.parameters(), lr=1e-4)

In [18]:
# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#   for batch in train_dataloader:
#     batch = {k:v.to(device) for k,v in batch.items()}
#     outputs = model(**batch)
#     loss = outputs.loss
#     loss.backward()

#     optimizer.step()
#     lr_scheduler.step()
#     optimizer.zero_grad()
#     progress_bar.update(1)

In [ ]:
model.save_pretrained("t5_question_generation_model")